# Importando los paquetes que vamos a usar

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
from time import time
import math
from scipy import stats


# Algoritmos de clusterizacion
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering

# Para reducir dimensionalidad
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Metricas de los clusters
from sklearn import metrics
from sklearn.metrics import pairwise_distances

# Para la anotacion de tipos
from typing import Callable, List, Dict, Tuple

# Carga de los datos

In [ ]:
base_path = "dataset/datos_hogar_2020.csv"
df_global = pd.read_csv(base_path)

# Funciones comunes a todos los casos de estudio

Definimos en esta sección las funciones que vamos a usar a lo largo de todos los casos de estudio:

In [ ]:
def drop_columns(df: pd.DataFrame, exclude_columns: List[str] = None) -> pd.DataFrame:
    """
    Dada una lista con columnas a excluir, devuelve un nuevo dataframe en el que 
    hemos quitado estas columnas. Si exclude_columns es None, no se hace nada.
    
    En ningun caso se modifica df, se hace una copia de este y se opera con 
    dicha copia
    """
    
    # Hacemos una copia del dataframe
    df_filtered = df.copy()
    
    # Excluimos las columnas en caso de que se haya pasado una lista no None
    if exclude_columns is not None:
        df_filtered = df_filtered.drop(columns = exclude_columns)
    
    # Devolvemos el dataframe copia filtrado
    return df_filtered

## Decorador para tomar el tiempo de ejecución de una función

In [ ]:
def time_func(func):
    """
    Decorador que toma una funcion func -> result y devuelve una funcion 
    func' -> result, ellapsed
    """
    
    # Definimos la funcion modificada como hemos especificado en la documentacion
    def modified_function(*args, **kwargs):
        init = time()
        result = func(*args, **kwargs)
        end = time()
        
        # Devolvemos el resultado junto con el tiempo que tardo en ejecutarse
        return result, end - init
    
    return modified_function

## Funciones para calcular clusters

In [ ]:
@time_func
def compute_kmeans(df: pd.DataFrame, number_clusters: int = 3) -> np.array:
    """
    Dado un dataframe con los datos a clusterizar, calcula el vector con las
    etiquetas de la clusterizacion kmeans
    """
    
    kmeans = KMeans(n_clusters = number_clusters, random_state=0).fit(df)
    return kmeans.labels_

@time_func
def compute_dbscan(df: pd.DataFrame, eps: float = 0.3, min_samples: int = 10) -> np.array:
    """
    Dado un dataframe con los datos a clusterizar, calcula el vector con las
    etiquetas de la clusterizacion dbscan
    """
    db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs = 2).fit(df)
    return db.labels_

@time_func
def compute_birch(df: pd.DataFrame, num_clusters: int = None, threshold: float = 0.5) -> np.array:
    """
    Dado un dataframe con los datos a clusterizar, calcula el vector con las
    etiquetas de la clusterizacion birch
    """
    
    brc = Birch(n_clusters=num_clusters, threshold = threshold)
    brc.fit(df)
    return brc.labels_

@time_func
def compute_mean_shift(df: pd.DataFrame, bandwidth: float = 2.0, max_iter: int = 300) -> np.array:
    """
    Dado un dataframe con los datos a clusterizar, calcula el vector con las
    etiquetas de la clusterizacion Mean Shift
    """

    clustering = MeanShift(bandwidth=bandwidth, max_iter = max_iter).fit(df)
    return clustering.labels_

@time_func
def compute_agglomerative_clustering(df: pd.DataFrame, n_clusters: int = None, linkage: str = "ward") -> np.array:
    """
    Dado un dataframe con los datos a clusterizar, calcula el vector con las
    etiquetas de la clusterizacion de forma jerarquica
    """
    
    clustering = AgglomerativeClustering(n_clusters = n_clusters, linkage = linkage).fit(df)
    return clustering.labels_

def add_clustering_labels(df: pd.DataFrame, clustering_function: Callable, parameters: dict(), new_label_name: str, exclude_columns: List[str] = None) -> Tuple[pd.DataFrame, float]:
    """
    Dado un dataframe y un algoritmo de clusterizacion con sus parametros, devuelve el dataframe al que
    hemos añadido una columna con las nuevas etiquetas del clustering. Con esto nos evitamos escribir
    esta funcion para cada tipo de algorito para clusterizar. Solo tenemos que cambiar ese parametro
    
    Podemos excluir columnas para que no se usen en la clusterizacion
    
    Devuelve un dataframe con la nueva columna de etiquetas añadida y con el tiempo que tardo en ejecutarse
    """
    
    # Filtramos las columnas del dataframe que se quieren exluir del computo
    df_filtered = drop_columns(df, exclude_columns)
    
    # Realizamos el calculo
    # Todas las funciones que calculan clusters llevan el decorador @time_func 
    # y por tanto devuelven cuanto tiempo an tardado 
    labels, ellapsed = clustering_function(df_filtered, **parameters)

    # Añadimos las etiquetas a la vez que recuperamos las columnas eliminadas
    new_df = df.copy()
    new_df[new_label_name] = labels
    
    return new_df, ellapsed

## Funciones para evaluar clusters

In [ ]:
def metric_silhouette(df: pd.DataFrame, labels: np.array, exclude_columns: List[str] = None) -> float:
    """Calcula el indice de silhouette de una clusterizacion"""
    
    # Excluimos las columnas si se ha especificado asi por parametro
    df_filtered = drop_columns(df, exclude_columns)
    
    # Devolvemos la metrica buscada
    return metrics.silhouette_score(df_filtered, labels, metric='euclidean')

def metric_calinski(df: pd.DataFrame, labels: np.array, exclude_columns: List[str] = None) -> float:
    """Calcula el indice de Calinski-Harabasz de una clusterizacion"""
    
    # Excluimos las columnas si se ha especificado asi por parametro
    df_filtered = drop_columns(df, exclude_columns)
    
    # Devolvemos la metrica buscada
    return metrics.calinski_harabasz_score(df_filtered, labels)

def metric_davies_bouldin(df: pd.DataFrame, labels: np.array, exclude_columns: List[str] = None) -> float:
    """Calcula el indice de Davies-Bouldin de una clusterizacion"""
    
    # Excluimos las columnas si se ha especificado asi por parametro
    df_filtered = drop_columns(df, exclude_columns)
    
    # Devolvemos la metrica buscada
    return metrics.davies_bouldin_score(df_filtered, labels)

def metric_elements_in_each_cluster(labels: np.array) -> Tuple[float, float]:
    """Calcula el numero medio de elementos por cada cluster, y la desviacion tipica del
    numero de elementos por cada cluster"""
    
    # Tomamos los valores unicos en el conjunto de etiquetas
    # Esto nos da las etiquetas de los clusters que hemos construido
    cluster_labels = set(labels)
    
    # Calculamos el numero de elementos que hay en cada cluster
    elements_in_each_cluster = []
    
    for label in cluster_labels:
        curr_labels = [el for el in labels if el == label]
        elements_in_each_cluster.append(len(curr_labels))
    
    
    # Calculamos las estadisticas usando numpy
    elements_in_each_cluster = np.array(elements_in_each_cluster)
    mean = np.mean(elements_in_each_cluster)
    std = np.std(elements_in_each_cluster)
    
    return mean, std

def metric_total_clusters(labels: np.array) -> int:
    """Calcula el numero total de clusters que hemos generado con un algoritmo"""
    
    # Tomamos los valores unicos en el conjunto de etiquetas
    # Esto nos da las etiquetas de los clusters que hemos construido
    cluster_labels = set(labels)
    
    # Con esto, solo tenemos que devolver el tamaño de dicho conjunto
    return len(cluster_labels)
    
def compute_clustering_metrics(df: pd.DataFrame, labels: np.array) -> Dict[str, float]:
    """
    Computa todas las metricas que tenemos sobre un clustering.
    
    @param df el dataframe, con solo los datos de entrada (quitar etiquetas y otros metadatos)
    @param labels las etiquetas obtenidas en un proceso de clustering
    
    @return metrics diccionario con pares "nombre_metrica": valor_metrica 
    """
               
    # Aplicamos todos los computos necesarios
    silhouette = metric_silhouette(df, labels)
    calinski = metric_calinski(df, labels)
    davies = metric_davies_bouldin(df, labels)
    total_clusters = metric_total_clusters(labels)
    elements_mean, elements_std = metric_elements_in_each_cluster(labels)
    
    # Devolvemos todas las metricas en un diccionario
    metrics = dict()
    metrics["silhouette"] = silhouette
    metrics["calinski"] = calinski
    metrics["davies"] = davies
    metrics["total_clusters"] = total_clusters
    metrics["elements_mean"] = elements_mean
    metrics["elements_std"] = elements_std
    
    return metrics

## Funciones para realizar visualizaciones

In [ ]:
def custom_pair_plot(df: pd.DataFrame, hue: str = None, exclude_columns: List[str] = None):
    """
    Muestra una grafica pairplot de seaborn. Podemos especificar una lista de
    columnas que exluir de esta grafica de forma comoda
    """
    df_filtered = df.copy()
    if exclude_columns is not None:
        df_filtered = df_filtered.drop(columns = exclude_columns)
        
    sns.pairplot(df_filtered, hue = hue)

## Funciones para normalizar dataframes y borrar outliers

In [ ]:
def standarize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizamos las columnas de un dataframe para que tengan media
    0 y desviacion tipica 1"""
    
    df_norm = df.copy()
    df_norm = (df_norm - df_norm.mean()) / df_norm.std()
    
    return df_norm

def normalize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizamos las columnas de un dataframe al rango [0, 1]"""
    
    df_norm = df.copy()
    df_norm = (df_norm - df_norm.min()) / (df_norm.max() - df_norm.min())
    
    return df_norm

def remove_outliers(df: pd.DataFrame, iqr_threshold: float = 3.0) -> pd.DataFrame:
    """
    Borra outliers quitando aquellas fila en las que tengamos un outlier en alguna columna
    Consideramos Outliers aquellos que se alejan de la media más de iqr_threshold veces la 
    desviacion tipica (normalmente se usa un valor 3.0)
    
    Se devuelve un dataframe nuevo, el original no se modifica
    """
    
    df_filtered = df.copy()
    return df_filtered[(np.abs(stats.zscore(df_filtered)) < 3).all(axis=1)]

def nan_to_zero(value):
    """Hace cero el valor si es None o NaN. En otro caso devuelve el valor"""
    
    if value is None or math.isnan(value):
        return 0.0
    
    # El valor no tiene que ser tratado, asi que se devuelve sin modificar
    return value

# Filtrado previo del dataset global

Definimos en variables todas las columnas que, al leer el documento explicando el *dataset*, nos han resultado de interés. Una vez hecho esto, filtramos el *dataset* global para quedarnos solo con esas columnas, pues el resto no las vamos a usar en ningún caso de estudio. Es más, muchas de estas columnas no las usaremos en ninguno de los tres casos de estudio.

In [ ]:
# Definimos las columnas con las que nos vamos a quedar
renta_disponible_total_hogar = "HY020"
renta_ganada_alquilando = "HY040N"
ayuda_para_vivienda = "HY070N"
inversiones_de_capital = "HY090N"
retraso_pago_hipoteca_o_alquiler = "HS011"
retraso_pago_luz_agua_gas = "HS021"
retraso_pago_prestamos = "HS031"
capacidad_hogar_llegar_fin_mes = "HS120"
ingresos_minimos_para_llegar_fin_mes = "HS130"
vandalismo_en_la_zona = "HS190"
comunidad_autonoma = "DB040"
tipo_de_vivienda = "HH010"
regimen_de_tenencia_vivienda = "HH021"
habitaciones_vivienda = "HH030"
pidio_ayuda_economica_familiares = "H39A_U"
pidio_ayuda_economica_entidad = "H39B_U"
corte_energia = "H38A_U"
numero_miembros_hogar = "HX040"
tipo_de_hogar = "HX060" # Variable numerica categorica con muchos valores posibles
hogar_en_riesgo_pobreza = "vhPobreza"
hogar_carencia_material_severa = "vhMATDEP"
proposito_prestamos = lambda i: f"HI100_{i}"
cuota_hipoteca = "cuotahip"
gasto_en_comida = "HC010"
gasto_en_comida_bebida_fuera_casa = "HC020"
gasto_transporte_publico = "HC030"
gasto_transporte_privado = "HC040"
gastos_vivienda_mensual = "HH070"
viviendas_adicionales = "HV020"
renta_disponible_restado_alquiler = "vhRentaAIa"

# Filtramos todo el dataframe global para usar solo variables que en una primera
# lectura de las variables hemos considerado interesantes
df_global = df_global[[
    renta_disponible_total_hogar,
    renta_ganada_alquilando,
    ayuda_para_vivienda,
    inversiones_de_capital,
    retraso_pago_hipoteca_o_alquiler,
    retraso_pago_luz_agua_gas,
    retraso_pago_prestamos,
    capacidad_hogar_llegar_fin_mes,
    ingresos_minimos_para_llegar_fin_mes,
    vandalismo_en_la_zona,
    tipo_de_vivienda,
    regimen_de_tenencia_vivienda,
    habitaciones_vivienda,   
    pidio_ayuda_economica_familiares,
    pidio_ayuda_economica_entidad,
    corte_energia,
    numero_miembros_hogar,
    tipo_de_hogar,
    hogar_en_riesgo_pobreza,
    hogar_carencia_material_severa,
    comunidad_autonoma,
    
    gasto_en_comida,
    gasto_en_comida_bebida_fuera_casa,
    gasto_transporte_publico,
    gasto_transporte_privado,
    gastos_vivienda_mensual,
    viviendas_adicionales,

    # TODO -- borrar los propositos porque no parecen muy relevantes
    proposito_prestamos(1),
    proposito_prestamos(2),
    proposito_prestamos(3),
    proposito_prestamos(4),
    proposito_prestamos(5),
    proposito_prestamos(6),
    proposito_prestamos(7),
    proposito_prestamos(8),
    proposito_prestamos(9),  
    
    cuota_hipoteca,
    renta_disponible_restado_alquiler,
]]

Veamos con qué tipo de datos nos hemos quedado:

In [ ]:
df_global.describe()

# Variables añadidas

También queremos usar variables agregadas usando las variables en bruto que ya tenemos en el dataset. 

Tenemos tres tipos de variable que nos indican si pidieron ayuda a algún ente, divididas en tres tipos de ente. Nos interesa solo si pidieron ayuda, no a quién

In [ ]:
def pidio_ayuda_condicion(ayuda_familiar, ayuda_entidad):
    """Comprueba si, dadas las columnas de ayuda familiar y ayuda de entidad,
    se pidio ayuda o no se pidio"""
    
    if ayuda_familiar == 1 or ayuda_entidad == 1:
        return True
    else:
        return False



# Añadimos la nueva columna usando la condicion que hemos definido previamente
pidio_ayuda = "pidio_ayuda"
df_global[pidio_ayuda] = df_global.apply(lambda row: pidio_ayuda_condicion(row[pidio_ayuda_economica_familiares], row[pidio_ayuda_economica_entidad]), axis = 1)

Añadimos una variable agregando todos los gastos en transporte:

In [ ]:
def sum_gastos(publico, privado):
    
    # Tratamos los valores None o NaN
    # Para poder sumar ambos valores
    publico = nan_to_zero(publico)
    privado = nan_to_zero(privado)
        
    return publico + privado

gasto_transporte_total = "gasto_transporte_total"
df_global[gasto_transporte_total] = df_global.apply(lambda row: sum_gastos(row[gasto_transporte_privado], row[gasto_transporte_publico]), axis = 1)

Añadimos una variable que recoja si ha habido retraso en cualquier tipo de pago:

In [ ]:
def retraso_pagando(retraso_hipoteca, retraso_luz_agua, retraso_prestamos):
    """Comprueba si hubo algun retraso en cualquier tipo de pago"""
    
    # Hubo retraso en la hipoteca
    if retraso_hipoteca is not None and retraso_hipoteca != 3:
        return 1
    
    # Hubo retraso en la luz o el agua
    if retraso_luz_agua is not None and retraso_luz_agua != 3:
        return 1
    
    # Hubo retraso en algun prestamo
    if retraso_prestamos is not None and retraso_prestamos != 3:
        return 1

    # No hubo retraso en ningun apartado
    return 0


# Añadimos la nueva columna usando la condicion que hemos definido previamente
retraso_pago = "retraso_pago"
df_global[retraso_pago] = df_global.apply(lambda row: retraso_pagando(row[retraso_pago_hipoteca_o_alquiler], row[retraso_pago_luz_agua_gas], row[retraso_pago_prestamos]), axis = 1)

Añadimos una variable en la que recojamos los ingresos menos los gastos totales:

In [ ]:
def calcular_ingresos_menos_gastos(renta_disponible_total_hogar, gasto_en_comida, gasto_en_comida_bebida_fuera_casa, gastos_vivienda_mensual, gasto_transporte_total):
    """Calcula la resta de todos los ingresos - todos los gastos"""
    
    # Hacemos 0.0 los valores None o NaN para poder realizar operaciones aritmeticas
    renta_disponible_total_hogar_val = nan_to_zero(renta_disponible_total_hogar)
    gasto_en_comida_val = nan_to_zero(gasto_en_comida)
    gasto_en_comida_bebida_fuera_casa_val = nan_to_zero(gasto_en_comida_bebida_fuera_casa)
    gastos_vivienda_mensual_val = nan_to_zero(gastos_vivienda_mensual)
    gasto_transporte_total_val = nan_to_zero(gasto_transporte_total)
    
    return renta_disponible_total_hogar_val - gasto_en_comida_val - gasto_en_comida_bebida_fuera_casa_val - gastos_vivienda_mensual_val - gasto_transporte_total_val


ingresos_menos_gastos = "ingresos_menos_gastos"
df_global[ingresos_menos_gastos] = df_global.apply(
    lambda row: calcular_ingresos_menos_gastos(row[renta_disponible_total_hogar], row[gasto_en_comida], row[gasto_en_comida_bebida_fuera_casa], row[gastos_vivienda_mensual], row[gasto_transporte_total]), 
    axis = 1)
df_global[ingresos_menos_gastos]

Añadimos el nombre real de las comunidades autonomas:

In [ ]:
def map_nombre_comunidad(code_name):
    """Transformamos el codigo de la comunidad autonoma a su nombre real"""
    map_code_to_name = {
        "ES11": "Galicia", 
        "ES12": "Principado de Asturias" , 
        "ES13": "Cantabria" , 
        "ES21": "País Vasco" , 
        "ES22": "Comunidad Foral de Navarra" , 
        "ES23": "La Rioja" , 
        "ES24": "Aragón" , 
        "ES30": "Madrid" , 
        "ES41": "Castilla y Leon" , 
        "ES42": "Castilla-La Mancha" , 
        "ES43": "Extremadura", 
        "ES51": "Cataluña" , 
        "ES52": "Comunidad Valenciana" , 
        "ES53": "Illes Balears" , 
        "ES61": "Andalucía" , 
        "ES62": "Región de Murcia" , 
        "ES63": "Ceuta" , 
        "ES64": "Melilla" , 
        "ES70": "Canarias" , 
        "ESZZ": "Externo" , 
    }
    
    return map_code_to_name[code_name]

df_global[comunidad_autonoma] = df_global.apply(lambda row: map_nombre_comunidad(row[comunidad_autonoma]), axis = 1)

# Caso de estudio 01

## Definición del caso de estudio

En este caso de estudio queremos poner nuestra atención en aquellas personas que tuvieron algún retraso en cualquier tipo de pago. Usamos la variable que hemos definido previamente como agregado de los dos tipos de *retraso en el pago*.

In [ ]:
# Tomamos los datos de aquellos hogares que pidieron ayuda economica
df_first = df_global[df_global[retraso_pago] == 1]

# Nos quedamos con las columnas que nos interesan para este caso de estudio
columnas_interesantes = [
    renta_disponible_total_hogar,
    ingresos_minimos_para_llegar_fin_mes,
    gasto_en_comida,
    gastos_vivienda_mensual,
    gasto_transporte_total,
]
df_first = df_first[columnas_interesantes]

# Renombramos algunas columnas
df_first = df_first.rename(columns = {
    renta_disponible_total_hogar: "renta_disponible_hogar",
    ingresos_minimos_para_llegar_fin_mes: "ingresos_minimos_para_llegar_fin_mes",
    gasto_en_comida: "gasto_en_comida",
    gastos_vivienda_mensual: "gastos_vivienda_mensual",
    gasto_transporte_total: "gasto_transporte_total",
})

## Preprocesado de los datos

Ahora que hemos seleccionado las variables, borramos aquellas filas en las que tengamos algún valor NaN:

In [ ]:
print(f"Antes: {df_first.shape}")
df_first = df_first.dropna()
print(f"Despues: {df_first.shape}")

Borramos los *outliers* usando $3 \cdot IQR$

In [ ]:
print(f"Antes: {df_first.shape}")
df_first = remove_outliers(df_first, 3.0)
print(f"Despues: {df_first.shape}")

Normalizamos todos los datos, pues tener variables en distintas escalas afecta enormemente a los algoritmos de clusterización:

In [ ]:
df_first = normalize_dataframe(df_first)
df_first

## Análisis exploratorio de datos

Antes de comenzar con el cálculo de *clusters*, realizamos un análisis exploratorio de datos previo. Empezamos mostrando las gráficas entre las variables con las que trabajamos, dos a dos. 

In [ ]:
sns.pairplot(df_first)

En un primer momento, consideramos usar las variables `renta_disponible_total_hogar` y `renta_disponible_restado_alquiler`. Sin embargo, como mostramos a continuación, están muy correladas, por lo que no aportan información interesante:

In [ ]:
sns.pairplot(df_global[[renta_disponible_total_hogar, renta_disponible_restado_alquiler]])

También calculábamos el valor de $total\_ingresos - total\_gastos$, pero vemos una gran correlación, como mostramos a continuación:

In [ ]:
sns.pairplot(df_global[[renta_disponible_total_hogar, ingresos_menos_gastos]])

## Aplicamos algoritmos de clustering

En todos los algoritmos estamos calculando los tiempos que tardan en ejecutarse. Esto lo guardaremos en el siguiente diccionario:

In [ ]:
ellapsed_times = dict()

Empezamos aplicando el algoritmo `k-means`:

In [ ]:
df_first, ellapsed = add_clustering_labels(
    df_first,
    compute_kmeans, 
    parameters = {"number_clusters": 4},
    new_label_name = "label_kmeans"
)

# Añado el tiempo consumido
ellapsed_times["label_kmeans"] = ellapsed

custom_pair_plot(df_first, hue = "label_kmeans")

Ahora aplicamos `dbscan` para la clusterización:

In [ ]:
df_first, ellapsed = add_clustering_labels(
    df_first,
    compute_dbscan,
    parameters = {"eps": 0.4, "min_samples": 5},
    new_label_name = "label_dbscan",
    exclude_columns = ["label_kmeans"]
) 

# Añado el tiempo consumido
ellapsed_times["label_dbscan"] = ellapsed

custom_pair_plot(
    df = df_first,
    hue = "label_dbscan",
    exclude_columns = ["label_kmeans"]
)

Ahora aplicamos `birch`:

In [ ]:
# TODO -- probar con num_clusters = None que es lo que pone en la pagina de ejemplo
# TODO -- quite eso porque tardaba demasiado
df_first, ellapsed = add_clustering_labels(
    df_first,
    compute_birch,
    parameters = {"num_clusters": 5, "threshold": 0.2},
    new_label_name = "label_birch",
    exclude_columns = ["label_kmeans", "label_dbscan"]
) 

# Añado el tiempo consumido
ellapsed_times["label_birch"] = ellapsed

custom_pair_plot(
    df = df_first,
    hue = "label_birch",
    exclude_columns = ["label_kmeans", "label_dbscan"]
)

Ahora aplicamos *Mean Shift*

In [ ]:
df_first, ellapsed = add_clustering_labels(
    df_first,
    compute_mean_shift,
    parameters = {"bandwidth": None, "max_iter": 1_000},
    new_label_name = "label_mean_shift",
    exclude_columns = ["label_kmeans", "label_dbscan", "label_birch"]
) 

# Añado el tiempo consumido
ellapsed_times["label_mean_shift"] = ellapsed

custom_pair_plot(
    df = df_first,
    hue = "label_mean_shift",
    exclude_columns = ["label_kmeans", "label_dbscan", "label_birch"]
)

Ahora aplicamos *clustering* jerárquico:

In [ ]:
df_first, ellapsed = add_clustering_labels(
    df_first,
    compute_agglomerative_clustering,
    parameters = {"n_clusters": 5, "linkage": "ward"},
    new_label_name = "label_agglomerative",
    exclude_columns = ["label_kmeans", "label_dbscan", "label_birch", "label_mean_shift"]
) 

# Añado el tiempo consumido
ellapsed_times["label_agglomerative"] = ellapsed

custom_pair_plot(
    df = df_first,
    hue = "label_agglomerative",
    exclude_columns = ["label_kmeans", "label_dbscan", "label_birch", "label_mean_shift"]
)

## Métricas de las clusterizaciones

Una vez que hemos aplicado los cinco algoritmos anteriores, recogemos las métricas de cada algoritmo para poder compararlas:

In [ ]:
# Todas las metricas que hemos construido
labels = ["label_kmeans", "label_dbscan", "label_birch", "label_mean_shift", "label_agglomerative"]

# Dataframe sin las columnas con las metricas
df_raw = drop_columns(df_first, labels)

# Diccionario que vamos a construir agregando todas las metricas
global_metrics = dict()

# Iteramos sobre cada una de las metricas
for label in labels:
    print(label)
    print("=" * 80)
    print("")
    
    # Tomamos las etiquetas del algoritmo actual y calculamos las metricas+
    curr_labels = df_first[label]
    curr_metrics = compute_clustering_metrics(df_raw, curr_labels)
    
    # Añadimos el tiempo de ejecucion para que se muestre con el resto de metricas
    curr_metrics["ellapsed_time (s)"] = ellapsed_times[label]
    
    # Mostramos dichas metricas
    pprint(curr_metrics)
    print("")
    print("")
    
    # Guardamos el diccionario con metricas en el nuevo diccionario con todas las metricas
    global_metrics[label] = curr_metrics

## Visualizando estas métricas

Vamos a mostrar gráficamente algunas de estas métricas de forma comparativa entre los algoritmos considerados. Nos basamos fuertemente en el diccionario con las métricas agregadas de distintos algoritmos para realizar estas visualizaciones. Empezamos comparando los tiempos de ejecución:

In [ ]:
# Etiquetas de los algoritmos que hemos usado
labels = ["label_kmeans", "label_dbscan", "label_birch", "label_mean_shift", "label_agglomerative"]

# Tomamos los tiempos de ejecucion
times = [global_metrics[label]["ellapsed_time (s)"] for label in labels]

# Ordenamos las dos listas basandonos en la de tiempos, para obtener una mejor visualizacion
times, labels = zip(*sorted(zip(times, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["time (s)"] = times

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "time (s)", y = "labels")

Veamos ahora el número de clusters que han generado los algoritmos:

In [ ]:
# Tomamos el numero de clusters
num_clusters = [global_metrics[label]["total_clusters"] for label in labels]

# Ordenamos las dos listas basandonos en la de numero de clusters, para obtener una mejor visualizacion
num_clusters, labels = zip(*sorted(zip(num_clusters, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["total_clusters"] = num_clusters

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "total_clusters", y = "labels")

Comparamos ahora los índices de Silhouette, que recordemos que queremos que sean lo más cercano a 1:

In [ ]:
silhouette = [global_metrics[label]["silhouette"] for label in labels]
silhouette, labels = zip(*sorted(zip(silhouette, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["silhouette"] = silhouette

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "silhouette", y = "labels")

Hacemos lo mismo con Calinski:

In [ ]:
calinski = [global_metrics[label]["calinski"] for label in labels]
calinski, labels = zip(*sorted(zip(calinski, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["calinski"] = calinski

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "calinski", y = "labels")

Hacemos lo mismo con Davies:

In [ ]:
davies = [global_metrics[label]["davies"] for label in labels]
davies, labels = zip(*sorted(zip(davies, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["davies"] = davies

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "davies", y = "labels")

Y veamos ahora las dos estadísticas sobre el número de elementos por cada cluster:

<!-- TODO -- juntar estas dos graficas en una sola -->

In [ ]:
elements_mean = [global_metrics[label]["elements_mean"] for label in labels]
elements_mean, labels = zip(*sorted(zip(elements_mean, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["elements_mean"] = elements_mean

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "elements_mean", y = "labels")

In [ ]:
elements_std = [global_metrics[label]["elements_std"] for label in labels]
elements_std, labels = zip(*sorted(zip(elements_std, labels)))

# Creamos un dataframe para mostrar usando seaborn
df_sns = pd.DataFrame()
df_sns["labels"] = labels
df_sns["elements_std"] = elements_std

# Mostramos la grafica buscada
sns.barplot(data = df_sns, x = "elements_std", y = "labels")

## Visualizando en 2D la clusterización

Usamos una técnica como `PCA` para reducir el conjunto de datos multidimensional a tener tan solo 2 dimensiones. A partir de aquí, mostramos gráficamente la clusterización que hemos realizado con cada uno de los algoritmos:

In [ ]:
# Reducimos la dimensionalidad del conjunto de datos
pca = PCA(2)

# Reducimos la dimensionalidad
df_lowdimension = pd.DataFrame(pca.fit_transform(df_raw))

# Renombramos las columnas
df_lowdimension = df_lowdimension.rename(columns = {
    0: "first_axis",
    1: "second_axis"
})

In [ ]:
# Por cada algoritmo, mostramos la clusterizacion realizada
for label in labels:
    print(f"Grafico usando las etiquetas de {label}")
    sns.scatterplot(data = df_lowdimension, x = "first_axis", y = "second_axis", hue = df_first[label])
    plt.show()

Realizamos lo mismo pero usando `TSNE` como algoritmo para reducir la dimensionalidad:

In [ ]:
# Reducimos la dimensionalidad
df_lowdimension = pd.DataFrame(TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(df_raw))

# Renombramos las columnas
df_lowdimension = df_lowdimension.rename(columns = {
    0: "first_axis",
    1: "second_axis"
})

In [ ]:
# Por cada algoritmo, mostramos la clusterizacion realizada
for label in labels:
    print(f"Grafico usando las etiquetas de {label}")
    sns.scatterplot(data = df_lowdimension, x = "first_axis", y = "second_axis", hue = df_first[label])
    plt.show()